Problem Statement.

There is an 8 x 8 chessboard containing n pieces (rooks, queens, or bishops). You are given a string array pieces of length n, where pieces[i] describes the type (rook, queen, or bishop) of the ith piece. In addition, you are given a 2D integer array positions also of length n, where positions[i] = [ri, ci] indicates that the ith piece is currently at the 1-based coordinate (ri, ci) on the chessboard.

When making a move for a piece, you choose a destination square that the piece will travel toward and stop on.

    A rook can only travel horizontally or vertically from (r, c) to the direction of (r+1, c), (r-1, c), (r, c+1), or (r, c-1).
    A queen can only travel horizontally, vertically, or diagonally from (r, c) to the direction of (r+1, c), (r-1, c), (r, c+1), (r, c-1), (r+1, c+1), (r+1, c-1), (r-1, c+1), (r-1, c-1).
    A bishop can only travel diagonally from (r, c) to the direction of (r+1, c+1), (r+1, c-1), (r-1, c+1), (r-1, c-1).

You must make a move for every piece on the board simultaneously. A move combination consists of all the moves performed on all the given pieces. Every second, each piece will instantaneously travel one square towards their destination if they are not already at it. All pieces start traveling at the 0th second. A move combination is invalid if, at a given time, two or more pieces occupy the same square.

Return the number of valid move combinations​​​​​.

Notes:

    No two pieces will start in the same square.
    You may choose the square a piece is already on as its destination.
    If two pieces are directly adjacent to each other, it is valid for them to move past each other and swap positions in one second.

 

Example 1:

Input: pieces = ["rook"], positions = [[1,1]]
Output: 15
Explanation: The image above shows the possible squares the piece can move to.

Example 2:

Input: pieces = ["queen"], positions = [[1,1]]
Output: 22
Explanation: The image above shows the possible squares the piece can move to.

Example 3:

Input: pieces = ["bishop"], positions = [[4,3]]
Output: 12
Explanation: The image above shows the possible squares the piece can move to.

 

Constraints:

    n == pieces.length
    n == positions.length
    1 <= n <= 4
    pieces only contains the strings "rook", "queen", and "bishop".
    There will be at most one queen on the chessboard.
    1 <= xi, yi <= 8
    Each positions[i] is distinct.

# Brute Force and Caching - O(29 ^ 4) runtime O(29 ^ 4) space

In [3]:
from typing import List
from functools import lru_cache

class Solution:
    def countCombinations(self, pieces: List[str], positions: List[List[int]]) -> int:
        moves = { "rook" : [ (1,0),(-1,0),(0,-1),(0,1) ],
                  "bishop" : [ (1,1),(-1,-1),(1,-1),(-1,1) ],
                  "queen" : [ (1,0),(-1,0),(0,-1),(0,1),(1,1),(-1,-1),(1,-1),(-1,1) ] }

        # bq -> blocked cells where we can't be. ( i, j, time ) as we care about time too
        # nbq -> same as above but it includes the current move
        # nxbq -> nbd and it adds the time until the end, as we will stay in this cell and wont let others pieces in
        
        @lru_cache(maxsize=None)
        def search( p, bq ) -> int:
            if p == len( pieces ):
                return 1
            
            res, t = 0, 0
            for m in moves[ pieces[p] ]:
                i, j = positions[p]
                # only start from initial position the first time.
                if t == 1:
                    i, j = i + m[0], j + m[1]
                nbq = bq
                    
                while i > 0 and i < 9 and j > 0 and j < 9:
                    if ( i, j, t ) in nbq:
                        break
                    nbq = nbq + (( i, j, t ),)
                    nxbq = nbq

                    # Figure out if we can stay in this cell until the end of the game
                    stay = True
                    for nt in range( t+1, 8 ):
                        if ( i, j, nt ) in bq:
                            stay = False
                            break
                        nxbq = nxbq + (( i, j, nt ),)

                    if stay:
                        res += search( p+1, nxbq )
                    i, j, t = i + m[0], j + m[1], t + 1
                
                i, j = positions[p]
                if ( i, j, 0 ) not in bq:
                    bq += (( i, j, 0 ),)
                t = 1
                    
            return res
        
        return search( 0, () )

In [4]:
instance = Solution()
instance.countCombinations( ["rook"],[[1, 1]])

15